In [1]:
from typing import Tuple
import os
import sys
import torch
import fire
import time
import json

from pathlib import Path

#from fairscale.nn.model_parallel.initialize import initialize_model_parallel

from llama import ModelArgs, Transformer, Tokenizer, LLaMA

In [2]:
from llama.generation import sample_top_p

### Load Model/Tokenizer

In [3]:
checkpoint = torch.load('../7B/consolidated.00.pth', map_location="cpu")

with open('../7B/params.json', "r") as f:
    params = json.loads(f.read())

model_args: ModelArgs = ModelArgs(
    max_seq_len=512, max_batch_size=1, **params
)
tokenizer = Tokenizer('../tokenizer.model')
model_args.vocab_size = tokenizer.n_words
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = Transformer(model_args)
torch.set_default_tensor_type(torch.FloatTensor)
model.load_state_dict(checkpoint,strict= False)

generator = LLaMA(model, tokenizer)

### Initialization

In [4]:
temperature: float = 0.8
top_p: float = 0.95
max_seq_len=512
max_batch_size=1

In [5]:
prompts = ['I believe the meaning of life is']

In [6]:
max_gen_len = 256

bsz = 1 
params = params
prompt_tokens = [tokenizer.encode(x, bos=True, eos=False) for x in prompts]
min_prompt_size = min([len(t) for t in prompt_tokens])
max_prompt_size = max([len(t) for t in prompt_tokens])

total_len = min(max_seq_len, max_gen_len + max_prompt_size)

tokens = torch.full((bsz, total_len),tokenizer.pad_id).cuda().long()

for k, t in enumerate(prompt_tokens):
    tokens[k, : len(t)] = torch.tensor(t).long()
input_text_mask = tokens != generator.tokenizer.pad_id
start_pos = min_prompt_size
prev_pos = 0

### Export

In [7]:
cur_pos = start_pos

dummy_input = torch.cat((tokens[:, prev_pos:cur_pos],torch.tensor([[prev_pos]]).cuda()), 1)

In [10]:
with torch.no_grad():
    model.eval()
    trace = torch.jit.trace(generator.model, dummy_input)
    torch.jit.save(trace, './full_llama.pt')

/root/Workspace/nopara_llama/llama/model.py:278: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  start_pos = input_tensor[:, -1].item()
/root/Workspace/nopara_llama/llama/model.py:87: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  xq_shape[-1] = int(xq_shape[-1]/2)
/root/Workspace/nopara_llama/llama/model.py:90: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize 

In [ ]:
decoded = []
for i, t in enumerate(tokens.tolist()):
    # cut to max gen len
    t = t[: len(prompt_tokens[i]) + max_gen_len]
    # cut to eos tok if any
    try:
        t = t[: t.index(generator.tokenizer.eos_id)]
    except ValueError:
        pass
    decoded.append(generator.tokenizer.decode(t))

decoded